<a href="https://colab.research.google.com/github/surya323-ma/chess.py/blob/main/chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Chess game for Google Colab
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets

class ChessGame:
    def __init__(self):
        self.board = self.initialize_board()
        self.current_player = 'white'
        self.game_over = False
        self.vs_computer = False
        self.selected_piece = None
        self.valid_moves = []

    def initialize_board(self):
        # Create initial chess board setup
        board = np.empty((8, 8), dtype=object)

        # Set up pawns
        board[1, :] = ['black_pawn'] * 8
        board[6, :] = ['white_pawn'] * 8

        # Set up other pieces
        pieces = ['rook', 'knight', 'bishop', 'queen', 'king', 'bishop', 'knight', 'rook']
        board[0, :] = ['black_' + piece for piece in pieces]
        board[7, :] = ['white_' + piece for piece in pieces]

        # Empty squares
        board[2:6, :] = 'empty'

        return board

    def render_board(self):
        clear_output(wait=True)

        # Create buttons for each square
        grid = widgets.GridBox(
            children=[],
            layout=widgets.Layout(
                grid_template_columns='repeat(8, 50px)',
                grid_template_rows='repeat(8, 50px)',
                grid_gap='1px'
            )
        )

        for row in range(8):
            for col in range(8):
                piece = self.board[row, col]
                color = '#f0d9b5' if (row + col) % 2 == 0 else '#b58863'

                if self.selected_piece and self.selected_piece == (row, col):
                    color = '#99ff99'  # Highlight selected piece

                if (row, col) in self.valid_moves:
                    color = '#ff9999'  # Highlight possible moves

                btn = widgets.Button(
                    description=self.get_piece_symbol(piece),
                    layout=widgets.Layout(width='100%', height='100%'),
                    style={'button_color': color}
                )

                # Assign callback with row and col information
                btn.row = row
                btn.col = col
                btn.on_click(self.handle_click)

                grid.children += (btn,)

        # Display game status and controls
        status_label = widgets.Label(value=f"Current player: {self.current_player}")
        game_mode = widgets.ToggleButtons(
            options=['Two Players', 'Vs Computer'],
            description='Game Mode:',
            value='Two Players'
        )
        game_mode.observe(self.change_game_mode, names='value')

        reset_btn = widgets.Button(description="Reset Game")
        reset_btn.on_click(self.reset_game)

        display(widgets.VBox([
            game_mode,
            widgets.HBox([status_label, reset_btn]),
            grid
        ]))

    def get_piece_symbol(self, piece):
        if piece == 'empty':
            return ''

        symbols = {
            'pawn': '♟',
            'rook': '♜',
            'knight': '♞',
            'bishop': '♝',
            'queen': '♛',
            'king': '♚'
        }

        color, piece_type = piece.split('_')
        symbol = symbols.get(piece_type, '')

        return symbol

    def handle_click(self, btn):
        if self.game_over:
            return

        row, col = btn.row, btn.col
        piece = self.board[row, col]

        # If no piece selected and clicked on own piece
        if self.selected_piece is None:
            if piece != 'empty' and piece.startswith(self.current_player):
                self.selected_piece = (row, col)
                self.valid_moves = self.get_valid_moves(row, col)
                self.render_board()
            return

        selected_row, selected_col = self.selected_piece

        # If clicked on same piece, deselect
        if (row, col) == (selected_row, selected_col):
            self.selected_piece = None
            self.valid_moves = []
            self.render_board()
            return

        # If clicked on valid move square
        if (row, col) in self.valid_moves:
            self.make_move(selected_row, selected_col, row, col)
            self.selected_piece = None
            self.valid_moves = []
            self.switch_player()

            if self.vs_computer and self.current_player == 'black' and not self.game_over:
                self.make_computer_move()
                self.switch_player()

            self.render_board()
            return

        # If clicked on another own piece
        if piece != 'empty' and piece.startswith(self.current_player):
            self.selected_piece = (row, col)
            self.valid_moves = self.get_valid_moves(row, col)
            self.render_board()
            return

        # Invalid selection
        self.selected_piece = None
        self.valid_moves = []
        self.render_board()

    def get_valid_moves(self, row, col):
        piece = self.board[row, col]
        if piece == 'empty':
            return []

        _, piece_type = piece.split('_')
        moves = []

        if piece_type == 'pawn':
            moves = self.get_pawn_moves(row, col)
        elif piece_type == 'rook':
            moves = self.get_rook_moves(row, col)
        elif piece_type == 'knight':
            moves = self.get_knight_moves(row, col)
        elif piece_type == 'bishop':
            moves = self.get_bishop_moves(row, col)
        elif piece_type == 'queen':
            moves = self.get_queen_moves(row, col)
        elif piece_type == 'king':
            moves = self.get_king_moves(row, col)

        return moves

    def get_pawn_moves(self, row, col):
        moves = []
        color, _ = self.board[row, col].split('_')
        direction = -1 if color == 'white' else 1

        # Forward move
        if 0 <= row + direction < 8 and self.board[row + direction, col] == 'empty':
            moves.append((row + direction, col))

            # Double move from starting position
            if (color == 'white' and row == 6) or (color == 'black' and row == 1):
                if self.board[row + 2*direction, col] == 'empty':
                    moves.append((row + 2*direction, col))

        # Capture moves
        for col_offset in [-1, 1]:
            new_col = col + col_offset
            if 0 <= new_col < 8:
                if 0 <= row + direction < 8:
                    target = self.board[row + direction, new_col]
                    if target != 'empty' and not target.startswith(color):
                        moves.append((row + direction, new_col))

        return moves

    def get_rook_moves(self, row, col):
        moves = []
        color, _ = self.board[row, col].split('_')

        # Horizontal and vertical moves
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

        for dr, dc in directions:
            for i in range(1, 8):
                new_row, new_col = row + i*dr, col + i*dc

                if not (0 <= new_row < 8 and 0 <= new_col < 8):
                    break

                target = self.board[new_row, new_col]

                if target == 'empty':
                    moves.append((new_row, new_col))
                else:
                    if not target.startswith(color):
                        moves.append((new_row, new_col))
                    break

        return moves

    def get_knight_moves(self, row, col):
        moves = []
        color, _ = self.board[row, col].split('_')

        knight_moves = [
            (2, 1), (1, 2), (-1, 2), (-2, 1),
            (-2, -1), (-1, -2), (1, -2), (2, -1)
        ]

        for dr, dc in knight_moves:
            new_row, new_col = row + dr, col + dc

            if 0 <= new_row < 8 and 0 <= new_col < 8:
                target = self.board[new_row, new_col]

                if target == 'empty' or not target.startswith(color):
                    moves.append((new_row, new_col))

        return moves

    def get_bishop_moves(self, row, col):
        moves = []
        color, _ = self.board[row, col].split('_')

        directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]

        for dr, dc in directions:
            for i in range(1, 8):
                new_row, new_col = row + i*dr, col + i*dc

                if not (0 <= new_row < 8 and 0 <= new_col < 8):
                    break

                target = self.board[new_row, new_col]

                if target == 'empty':
                    moves.append((new_row, new_col))
                else:
                    if not target.startswith(color):
                        moves.append((new_row, new_col))
                    break

        return moves

    def get_queen_moves(self, row, col):
        # Queen moves are combination of rook and bishop
        return self.get_rook_moves(row, col) + self.get_bishop_moves(row, col)

    def get_king_moves(self, row, col):
        moves = []
        color, _ = self.board[row, col].split('_')

        king_moves = [
            (1, 0), (-1, 0), (0, 1), (0, -1),
            (1, 1), (1, -1), (-1, 1), (-1, -1)
        ]

        for dr, dc in king_moves:
            new_row, new_col = row + dr, col + dc

            if 0 <= new_row < 8 and 0 <= new_col < 8:
                target = self.board[new_row, new_col]

                if target == 'empty' or not target.startswith(color):
                    moves.append((new_row, new_col))

        return moves

    def make_move(self, from_row, from_col, to_row, to_col):
        piece = self.board[from_row, from_col]
        self.board[to_row, to_col] = piece
        self.board[from_row, from_col] = 'empty'

        # Check for pawn promotion
        if 'pawn' in piece and (to_row == 0 or to_row == 7):
            self.board[to_row, to_col] = piece.replace('pawn', 'queen')

        # Check for game over (simplified - just check if king is captured)
        opponent_color = 'black' if piece.startswith('white') else 'white'
        king_exists = any(opponent_color + '_king' in cell for cell in self.board.flatten())

        if not king_exists:
            self.game_over = True

    def make_computer_move(self):
        # Simple AI: for each piece, get all valid moves and pick randomly
        import random

        computer_pieces = []
        for row in range(8):
            for col in range(8):
                piece = self.board[row, col]
                if piece != 'empty' and piece.startswith('black'):
                    moves = self.get_valid_moves(row, col)
                    if moves:
                        computer_pieces.append((row, col, moves))

        if computer_pieces:
            src_row, src_col, moves = random.choice(computer_pieces)
            dst_row, dst_col = random.choice(moves)
            self.make_move(src_row, src_col, dst_row, dst_col)

    def switch_player(self):
        self.current_player = 'black' if self.current_player == 'white' else 'white'

    def change_game_mode(self, change):
        self.vs_computer = change['new'] == 'Vs Computer'
        self.reset_game()

    def reset_game(self, btn=None):
        self.board = self.initialize_board()
        self.current_player = 'white'
        self.game_over = False
        self.selected_piece = None
        self.valid_moves = []
        self.render_board()

# Start the game
game = ChessGame()
game.render_board()
